<a href="https://colab.research.google.com/github/jangvu/Project_by_me/blob/main/LSTM_alice_in_wonderland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!ls "/content/drive/My Drive/Data"

101_ObjectCategories.tar.gz  Annotations.tar  isbi-datasets.zip  NFLX.csv
alice_in_wonderland.txt      candidate_test   NEU-DET.zip


In [15]:
FILE_PATH = '/content/drive/My Drive/Data/alice_in_wonderland.txt'

In [16]:
raw_text = open(FILE_PATH).read().lower()


In [17]:
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer

Take the set of characters in the texts

In [18]:
chars = sorted(set(raw_text))

Using a dictionary 59 to transcript the story's characters -> each time a char appears 1 in key of that char and 0 for the rest

In [19]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '3': 12, ':': 13, ';': 14, '?': 15, '[': 16, ']': 17, '_': 18, '`': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'k': 30, 'l': 31, 'm': 32, 'n': 33, 'o': 34, 'p': 35, 'q': 36, 'r': 37, 's': 38, 't': 39, 'u': 40, 'v': 41, 'w': 42, 'x': 43, 'y': 44, 'z': 45}


But we dont need to use all chars because some chars dont affect the sentences like *, #, $, ... => make new chars 

In [20]:
import string
string.ascii_lowercase
chars_new = list(string.ascii_lowercase) + ['0', '.', ',', ' ', '!', '?', 'unk']
chars_to_int = dict((v, k) for k, v in enumerate(chars_new))
int_to_chars = dict((k, v) for k, v in enumerate(chars_new))

print(chars_to_int)
print(int_to_chars)

n_chars = len(raw_text)
n_vocab = len(chars_new)
print('Total characters: ', n_chars)
print('Total Vocab: ', n_vocab)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, '0': 26, '.': 27, ',': 28, ' ': 29, '!': 30, '?': 31, 'unk': 32}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z', 26: '0', 27: '.', 28: ',', 29: ' ', 30: '!', 31: '?', 32: 'unk'}
Total characters:  148574
Total Vocab:  33


In [21]:
def _encode_sen(text):
  text = text.lower()
  sen_vec = []
  for letter in text:
    if letter in chars_new:
      idx = chars_to_int[letter]
    else:
      idx = chars_to_int['unk']
    sen_vec.append(idx)
  return sen_vec

In [22]:
def _decode_sen(vec):
  text = []
  for idx in vec:
    letter = int_to_chars[idx]
    text.append(letter)
  text = ''.join(text)
  return text

In [23]:
a = _encode_sen('Alice in wonderland. #')
print('Encode')
print(a)
b = _decode_sen(a)
print('Decode')
print(b)

Encode
[0, 11, 8, 2, 4, 29, 8, 13, 29, 22, 14, 13, 3, 4, 17, 11, 0, 13, 3, 27, 29, 32]
Decode
alice in wonderland. unk


Creating Window to slide through text 

In [24]:
def window(text, step = 1, window_size = 10, test = False,):
  X_train = []
  y_train = []
  if test == True:
    for i in range(len(text) - window_size+1):
      X_train.append(_encode_sen(text[i:i+window_size]))
  else:
    for i in range(len(text) - window_size):
      X_train.append(_encode_sen(text[i:i+window_size]))
      y_train.append(_encode_sen(text[i+window_size]))
  return X_train, y_train




In [25]:
data_X, data_Y = window(raw_text)

INPUT_SHAPE has form (samples, time_steps, features). So in this problem samples = number of windows, time_steps = length of window, features is dimension of window (we using 1D in this)

In [26]:
num_samples = len(data_X)
window_size = 10

In [27]:
X_train = np.asarray(data_X).reshape(num_samples,window_size,1)
# Normalize
X_train = X_train / float(n_vocab)

#labelBinarizer = LabelBinarizer()
#y_train = labelBinarizer.fit_transform(np.asarray(data_Y))

#One Hot Code vector
y_train = np_utils.to_categorical(data_Y)

print('X_train shape:')
print(X_train.shape)
print('y_train shape:')
print(y_train.shape)


X_train shape:
(148564, 10, 1)
y_train shape:
(148564, 33)


In [28]:
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
plt.figure(figsize = (10, 5))
#sn.countplot(np.asarray(data_Y))
#plt.xticks(np.arange(32),np.array(chars_new))


<Figure size 720x360 with 0 Axes>

<Figure size 720x360 with 0 Axes>

**Creating Model**

In [29]:
import tensorflow as tf
from tensorflow import keras

In [30]:
model_LSTM = keras.models.Sequential()
model_LSTM.add(keras.layers.LSTM(256,input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences=True))
model_LSTM.add(keras.layers.LSTM(128))
model_LSTM.add(keras.layers.Dropout(0.2))
model_LSTM.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model_LSTM.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 256)           264192    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 33)                4257      
Total params: 465,569
Trainable params: 465,569
Non-trainable params: 0
_________________________________________________________________


In [31]:
filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callback_list = [checkpoint]

In [32]:
model_LSTM.fit(X_train, y_train, epochs = 50, batch_size = 128, validation_split=0.33, callbacks = callback_list, verbose = 1)

Epoch 1/50
778/778 [==============================] - 39s 7ms/step - loss: 2.9551 - accuracy: 0.1949 - val_loss: 2.7106 - val_accuracy: 0.2274
Epoch 2/50
778/778 [==============================] - 5s 6ms/step - loss: 2.6738 - accuracy: 0.2411 - val_loss: 2.5604 - val_accuracy: 0.2715
Epoch 3/50
778/778 [==============================] - 5s 6ms/step - loss: 2.5079 - accuracy: 0.2829 - val_loss: 2.3921 - val_accuracy: 0.3085
Epoch 4/50
778/778 [==============================] - 5s 6ms/step - loss: 2.3740 - accuracy: 0.3148 - val_loss: 2.2923 - val_accuracy: 0.3418
Epoch 5/50
778/778 [==============================] - 5s 6ms/step - loss: 2.2602 - accuracy: 0.3451 - val_loss: 2.2003 - val_accuracy: 0.3654
Epoch 6/50
778/778 [==============================] - 5s 6ms/step - loss: 2.1636 - accuracy: 0.3695 - val_loss: 2.1319 - val_accuracy: 0.3867
Epoch 7/50
778/778 [==============================] - 5s 6ms/step - loss: 2.0852 - accuracy: 0.3952 - val_loss: 2.0881 - val_accuracy: 0.3994
Epoch

**Predicting next char**

In [56]:
base_word = 'Alice was beginning to get very tired of sitting by her sister on the bank'
X_test, y_test = window(base_word,test=True)

test_samples = len(X_test)
X_test = np.asarray(X_test) / float(n_vocab)
X_test = X_test.reshape(test_samples,10,1)
y_prob_1 = model_LSTM.predict(X_test)
int_to_chars[np.argmax(y_prob_1, axis = 1)[-1]]

','

**Predicting next 10 chars**

In [59]:
def _predict_let(text, len_pred = 1,model = model_LSTM):
    for i in range(len_pred):
        X_test, y_test = window(text,test=True)
        test_samples = len(X_test)
        print(test_samples)
        X_test = np.asarray(X_test) / float(n_vocab)
        X_test = X_test.reshape(test_samples,10,1)
        y_prob = model.predict(X_test)
        y_let = int_to_chars[np.argmax(y_prob, axis = 1)[-1]]
        print(y_let)
        text = text + y_let
        print(text)
    return text

In [74]:
base_word = 'Alice was beginning to get very tired of sitting by her'
y_demo = _predict_let(base_word, len_pred=10)

46
 
Alice was beginning to get very tired of sitting by her 
47
s
Alice was beginning to get very tired of sitting by her s
48
i
Alice was beginning to get very tired of sitting by her si
49
s
Alice was beginning to get very tired of sitting by her sis
50
t
Alice was beginning to get very tired of sitting by her sist
51
e
Alice was beginning to get very tired of sitting by her siste
52
r
Alice was beginning to get very tired of sitting by her sister
53
 
Alice was beginning to get very tired of sitting by her sister 
54
w
Alice was beginning to get very tired of sitting by her sister w
55
a
Alice was beginning to get very tired of sitting by her sister wa


**Bidirectional LSTM**

In [71]:
learning_rate = 0.001
optimizer = keras.optimizers.Adam(lr=learning_rate)

model = keras.models.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, activation="tanh"),input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(y_train.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 256)               133120    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 33)                8481      
Total params: 141,601
Trainable params: 141,601
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_split=0.1, callbacks = callback_list, verbose = 1, shuffle = True)

Epoch 1/10
4179/4179 [==============================] - 25s 5ms/step - loss: 2.8481 - accuracy: 0.2074 - val_loss: 2.5802 - val_accuracy: 0.2703
Epoch 2/10
4179/4179 [==============================] - 22s 5ms/step - loss: 2.5405 - accuracy: 0.2758 - val_loss: 2.3856 - val_accuracy: 0.3141
Epoch 3/10
4179/4179 [==============================] - 22s 5ms/step - loss: 2.3348 - accuracy: 0.3244 - val_loss: 2.2323 - val_accuracy: 0.3482
Epoch 4/10
4179/4179 [==============================] - 22s 5ms/step - loss: 2.1698 - accuracy: 0.3626 - val_loss: 2.0506 - val_accuracy: 0.4006
Epoch 5/10
4179/4179 [==============================] - 22s 5ms/step - loss: 2.0715 - accuracy: 0.3860 - val_loss: 2.0585 - val_accuracy: 0.3954
Epoch 6/10
4179/4179 [==============================] - 22s 5ms/step - loss: 1.9910 - accuracy: 0.4080 - val_loss: 1.9044 - val_accuracy: 0.4329
Epoch 7/10
4179/4179 [==============================] - 22s 5ms/step - loss: 1.9064 - accuracy: 0.4315 - val_loss: 1.8555 - val_ac

In [75]:
base_word = 'Alice was beginning to get very tired of sitting by her'
y_prob_2 = _predict_let(base_word, len_pred=10, model = model)

46
 
Alice was beginning to get very tired of sitting by her 
47
t
Alice was beginning to get very tired of sitting by her t
48
o
Alice was beginning to get very tired of sitting by her to
49
 
Alice was beginning to get very tired of sitting by her to 
50
t
Alice was beginning to get very tired of sitting by her to t
51
h
Alice was beginning to get very tired of sitting by her to th
52
e
Alice was beginning to get very tired of sitting by her to the
53
 
Alice was beginning to get very tired of sitting by her to the 
54
t
Alice was beginning to get very tired of sitting by her to the t
55
h
Alice was beginning to get very tired of sitting by her to the th
